## SATCEN-01-01-01 Sentinel-2 Vegetation and Water Thematic Index application

In [155]:
service = dict([('title', 'NDVI NDWI & Cloud Coverage Filtering'),
                ('abstract', 'Sentinel-2 NDVI NDWI'),
                ('id', 'ewf-satcen-01-01-01')])

In [156]:
resolution = dict([('id', 'resolution'),
               ('value', '60'),
               ('title', 'Spatial resolution'),
               ('abstract', 'Spatial resolution in meters (Only 60)')])

In [157]:
plot_quicklooks = dict([('id', 'plot'),
               ('value', 'False'),
               ('title', 'Boolean to add quicklooks to notebook'),
               ('abstract', 'Boolean to add quicklooks to notebook')])

In [187]:
wkt = dict([('id', 'pa_wkt'),
               ('value','POLYGON ((-7.671564606238091 37.30020596254354, -7.643312660718724 37.39964708590648, -7.601223974855643 37.54690948118862, -7.559078042948089 37.69414724133335, -7.546671889395634 37.73733941088647, -6.819372833545006 37.72644227043538, -6.837964713309738 37.13737011293922, -7.673831189524798 37.14961656163176, -7.671564606238091 37.30020596254354))'),
               ('title', 'Protected Area wkt'),
               ('abstract', 'Protected Area wkt')])



In [188]:
data_path = '/workspace/data'

In [189]:
input_identifier = 'S2A_MSIL2A_20170909T110651_N0205_R137_T29SPB_20170909T111217'

In [190]:

input_reference = 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL2A_20170909T110651_N0205_R137_T29SPB_20170909T111217'


In [191]:
import os
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import snappy
import dateutil.parser as parser
import gc
from datetime import datetime
import matplotlib
import matplotlib.colors as colors
from PIL import Image
from PIL import ImageDraw
from os.path import basename
import gdal
import osr


from os.path import exists
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
from struct import unpack
from sys import argv
from sys import exit

%matplotlib inline

In [192]:
s2prd = "%s/%s/%s.SAFE/MTD_MSIL2A.xml" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s2prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()


In [195]:
product_date = parser.parse(product.getStartTime().toString()).date()

In [196]:
output_date = '%s%02d%02d' % (product_date.year, product_date.month, product_date.day)

In [193]:
print('Bands:  %s' % (list(band_names)))


Bands:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'quality_aot', 'quality_wvp', 'quality_cloud_confidence', 'quality_snow_confidence', 'quality_scene_classification', 'view_zenith_mean', 'view_azimuth_mean', 'sun_zenith', 'sun_azimuth', 'view_zenith_B1', 'view_azimuth_B1', 'view_zenith_B2', 'view_azimuth_B2', 'view_zenith_B3', 'view_azimuth_B3', 'view_zenith_B4', 'view_azimuth_B4', 'view_zenith_B5', 'view_azimuth_B5', 'view_zenith_B6', 'view_azimuth_B6', 'view_zenith_B7', 'view_azimuth_B7', 'view_zenith_B8', 'view_azimuth_B8', 'view_zenith_B8A', 'view_azimuth_B8A', 'view_zenith_B9', 'view_azimuth_B9', 'view_zenith_B10', 'view_azimuth_B10', 'view_zenith_B11', 'view_azimuth_B11', 'view_zenith_B12', 'view_azimuth_B12']


In [197]:
#if resolution['value'] == '10':
#    reference_band = 'B4'

#if resolution['value'] == '20':
#    reference_band = 'B5'
    
#if resolution['value'] == '60':
reference_band = 'B1'   

In [198]:


snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')


    
parameters = HashMap()
parameters.put('referenceBand', reference_band)

#resample B1
product = snappy.GPF.createProduct('Resample', parameters, product)


    

In [199]:
flag_expr = dict([('id', 'flag_expr'),
               ('value', '( saturated_l1a_B4 or scl_water )'),
               ('title', 'Flag expression for pixel exclusion'),
               ('abstract', 'Flag expression for pixel exclusion (e.g. saturated_l1a_B4 will exclude pixels having the flag saturated_l1a_B4 set)')])

## ndvi & ndwi computation

In [200]:
if not flag_expr['value']:
    ndvi_expr = '(B8 + B4) != 0 ? 10000 + ((B8 - B4)/(B8 + B4)) * 10000 : 30000'
    ndwi_expr='(B8+B11)!=0? 10000+((B8-B11)/(B8+B11))*10000 :30000'
else:
    ndvi_expr = '! %s and (B8 + B4) != 0 ? 10000 + ((B8 - B4)/(B8 + B4)) * 10000 : 30000' % flag_expr['value']
    ndwi_expr = '! %s and (B8 + B11) != 0 ? 10000 + ((B8 - B11)/(B8 + B11)) * 10000 : 30000' % flag_expr['value']


In [194]:
displayed_Bands=['ndwi','ndvi','quality_cloud_confidence','quality_snow_confidence', 'quality_scene_classification','B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

print(len(displayed_Bands))
print(displayed_Bands)





17
['ndwi', 'ndvi', 'quality_cloud_confidence', 'quality_snow_confidence', 'quality_scene_classification', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']


## Cloud Coverage Analysis

In [201]:
percentage_threshold = dict([('id', 'percentage_threshold'),
                           ('value', '20.0'),
                           ('title', 'Cloud percentage threshold'),
                           ('abstract', 'Cloud percentage threshold')])

In [202]:
HashMap = snappy.jpy.get_type('java.util.HashMap')

BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBand0 = BandDescriptor()
targetBand0.name = 'cloud_mask'
targetBand0.type = 'uint16'
targetBand0.expression = 'opaque_clouds_60m'


targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 1)
targetBands[0] = targetBand0

 
parameters = HashMap()
parameters.put('targetBands', targetBands)

cloud_mask = snappy.GPF.createProduct('BandMaths', parameters, product)


In [203]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt['value'])





parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)
    
cloud_mask_geo = snappy.GPF.createProduct('Subset', parameters, cloud_mask)

mask_geo_output_name = '%s_MASK_%s.tif' % (name, '60')
snappy.ProductIO.writeProduct(cloud_mask_geo, mask_geo_output_name,'GeoTIFF')

In [204]:
import gdalnumeric
raster_file = gdalnumeric.LoadFile(mask_geo_output_name)
print raster_file.min(), raster_file.max()
pixel_count_cloud_geo = (raster_file == 255).sum()  # for pixel value = 1
print pixel_count_cloud_geo
cloud_percent =  float(pixel_count_cloud_geo) / float(raster_file.size) * 100.0

0 255
577750


In [205]:
cloud_percent

42.63768411140188

In [209]:
if cloud_percent >= float(percentage_threshold['value']):
    
    snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

    HashMap = snappy.jpy.get_type('java.util.HashMap')

    WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
    
    geom = WKTReader().read(wkt['value'])
    
    subsets = []
    geotiffs = []
    
    for band in list(band_names)[0:16]:

        BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

        parameters = HashMap()
        parameters.put('referenceBand', band)

        resample = snappy.GPF.createProduct('Resample', parameters, product)

        parameters = HashMap()
        parameters.put('copyMetadata', True)
        parameters.put('sourceBands', band)

        subset = snappy.GPF.createProduct('Subset', parameters, resample)

        parameters = HashMap()
        parameters.put('copyMetadata', True)
        parameters.put('geoRegion', geom)

        subset_geo = snappy.GPF.createProduct('Subset', parameters, subset)

        subsets.append(subset_geo)

        #output_name = 'S2A_USER_PRD_MSIL1C_%s_%s_CROP_%s.tif' % (output_date, pa_code['value'], band)
        output_name = '%s_PA_CROP_%s.tif' % (name, band)
        geotiffs.append(output_name)

    snappy.ProductIO.writeProduct(subset_geo, output_name, 'GeoTIFF')

## Target Bands : B1/B12 & ndvi & ndwi & Cloud Mask


In [78]:
HashMap = snappy.jpy.get_type('java.util.HashMap')

BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')


targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(displayed_Bands))
for j in range(0,len(displayed_Bands)):
     Disp_target = BandDescriptor()
     Disp_target.name = displayed_Bands[j]
     Disp_target.type = 'uint16'
     if displayed_Bands[j]=='ndvi':
        Disp_target.expression =ndvi_expr
     elif displayed_Bands[j]=='ndwi':
        Disp_target.expression =ndwi_expr
     else:  
        Disp_target.expression = displayed_Bands[i]
     targetBands[j] = Disp_target
parameters = HashMap()
parameters.put('targetBands', targetBands)

ndvi_ndwi = snappy.GPF.createProduct('BandMaths', parameters, product)

B12


In [79]:
product = None



gc.collect()

4

#### Write the geotiff

In [80]:
output_name = '%s_NDVI_NDWI' %input_identifier

In [81]:
snappy.ProductIO.writeProduct(ndvi_ndwi, output_name + '.tif', 'GeoTIFF')

### EOP Metadata

In [82]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % opt)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    orbit_number.text = metadata['orbitNumber']
    wrs_longitude_grid.text = metadata['wrsLongitudeGrid']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)




## Create EOP XML file

In [83]:
import cioppy
ciop = cioppy.Cioppy()
from shapely.wkt import loads
import lxml.etree as etree


search = ciop.search(end_point=input_reference,
                     params=[],
                     output_fields='enclosure,identifier,startdate,enddate,wkt,orbitNumber,orbitDirection,wrsLongitudeGrid',
                     model='EOP')

search[0]['identifier'] = output_name.replace('.tif', '')

eop_xml = output_name.replace('.tif', '.xml')
with open(eop_xml, 'wb') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    file.write(eop_metadata(search[0]))

